In [47]:
import os
import datetime
import json
import numpy as np
import pandas as pd

timestamp_conversion_factor = 10**9
export_directory = "/Users/Julian/Documents/WorkDocuments/Z24-S101-mini-project/S101-and-Z24-demonstrator/Z24_bridge_KUL/channel-data-json"

first_day = datetime.datetime(1997,11,10,14)
day_offset = {"A": -2, "B": -1, "C": 0, "D": 1, "E": 2, "F": 3, "G": 4}

main_dir = "/Users/Julian/Documents/WorkDocuments/Z24-S101-mini-project/S101-and-Z24-demonstrator/Z24_bridge_KUL/accelerations"

sub_dirs = ["/Z24ems1", "/Z24ems2", "/Z24ems3"]

# find data directories in first sub directory
data_dir = next(os.walk(f"{main_dir}{sub_dirs[0]}"))[1]
# print week, day, hour info for the data directory in question
week = data_dir[0][0:2]
day = data_dir[0][2]
hour = data_dir[0][3:]
# print(f"week: {week}\nday: {day}\nhour: {hour}")
# calculate the day and time that the data was collected on
days_into_project = datetime.timedelta(hours = int(hour) - 14,days = day_offset[day], weeks = int(week) - 1)
new_date = first_day + days_into_project
# print(new_date)

# find data files within the first data directory in the first sub directory
data_files = next(os.walk(f"{main_dir}{sub_dirs[0]}/{data_dir[0]}"))[2]
# find only the .aaa files and exclude the car log as its name is over 11 characters long
acceleration_files = [file for file in data_files if file.endswith(".aaa") and len(file) == 11]
print(acceleration_files)
channel_values = {}
channel_units = ["g"] * len(acceleration_files)
for j, acc_file in enumerate(acceleration_files):
	with open(f"{main_dir}{sub_dirs[0]}/{data_dir[0]}/{acc_file}") as file:
		# print(f"current file: {main_dir}{sub_dirs[0]}/{data_dir[0]}/{acc_file}")
		channel_number = acc_file[5:7]
		channel_values[channel_number] = []
		# print(f"channel number: {channel_number}")
		sample_time_offsets = []
		sample_values = []
		for i, line in enumerate(file):
			if i == 2:
				time_interval = float(line)
				# print(f"time interval: {time_interval}")
			# temporarily changed from 65539 to 10 to make it easier to debug
			elif 8 > i >= 3:
				sample_number = i-2
				sample_time_offsets.append(datetime.timedelta(seconds = sample_number * time_interval))
				sample_values.append(float(line)) 
			# if i == 65538:
				# print(f"final sample (number {sample_number}) is: {float(line)}")
			if line.startswith("Segment #1 Start"):
				segment_start_str = line.replace("Segment #1 Start :","").replace("\n", "")
				segment_start_date_time = datetime.datetime.strptime(segment_start_str, "%a %b %d %H:%M:%S %Y")
				# print(f"Segment #1 Start :{segment_start_date_time}")
		channel_values[channel_number] = sample_values
	if j == 0:
		# assuming that all samples are taken at the same across the channels
		sample_times = [segment_start_date_time + time_offset for time_offset in sample_time_offsets]
		sample_timestamps = [datetime.datetime.timestamp(sample_time) for sample_time in sample_times]
		reformatted_sample_timestamps = [int(sample_timestamp * timestamp_conversion_factor) for sample_timestamp in sample_timestamps]
		# print(reformatted_sample_timestamps)

all_channels_dataframe = pd.DataFrame(channel_values, reformatted_sample_timestamps)
# print(all_channels_dataframe)

for index, row in all_channels_dataframe.iterrows():
	# print(index)
	channels = []
	for channel_name in channel_values.keys():
		channel_object = {
						"name": f"channel-{channel_name}",
						"type": "acceleration",
						"unit": "g",
						"value": np.format_float_positional(row[channel_name], trim='-')
					}
		channels.append(channel_object)

	output_json = {
				"name": "Z24",
				"population": "realbridges",
				"timestamp": index,
				"channels": channels
			}

	with open (f"{export_directory}/Z24-{index}.json", "w") as outfile:
		json.dump(output_json, outfile, indent=4)


['07E0105.aaa', '07E0110.aaa', '07E0112.aaa', '07E0106.aaa', '07E0107.aaa', '07E0103.aaa', '07E0116.aaa', '07E0114.aaa']
<class 'list'>


TypeError: Object of type Timestamp is not JSON serializable